<a href="https://colab.research.google.com/github/bouvaw24/Shell/blob/main/Copy_of_gpt_llm_trainer_v2%2C_with_GPT_3_5_Fine_Tuning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Describe your model -> fine-tuned GPT-3.5
By Matt Shumer (https://twitter.com/mattshumer_)

The goal of this notebook is to experiment with a new way to make it very easy to build a task-specific model for your use-case.

First, use the best GPU available (go to Runtime -> change runtime type)

To create your model, just go to the first code cell, and describe the model you want to build in the prompt. Be descriptive and clear.

Select a temperature (high=creative, low=precise), and the number of training examples to generate to train the model. From there, just run all the cells.

You can change the model you want to fine-tune by changing `model_name` in the `Define Hyperparameters` cell.

I adjusted his code so that it would work with the newer update of Openai

#Data generation step

Write your prompt here. Make it as descriptive as possible!

Then, choose the temperature (between 0 and 1) to use when generating data. Lower values are great for precise tasks, like writing code, whereas larger values are better for creative tasks, like writing stories.

Finally, choose how many examples you want to generate. The more you generate, a) the longer it takes and b) the more expensive data generation will be. But generally, more examples will lead to a higher-quality model. 100 is usually the minimum to start.

In [ ]:
prompt = "Insert your prompt here" #make it as specific as possible to get the desired prompts and responses to train the model on
temperature = .4
number_of_examples = 25

Run this to generate the dataset.

In [ ]:
!pip install openai tenacity

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 221.4/221.4 kB 2.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.0/75.0 kB 5.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.9/76.9 kB 8.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 7.0 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
llmx 0.0.15a0 requires cohere, which is not installed.
llmx 0.0.15a0 requires tiktoken, which is not installed.


In [ ]:
!pip install openai --upgrade



In [ ]:
import os
from openai import OpenAI, Completion
import random

os.environ['OPENAI_API_KEY'] = "Insert API Key Here"
api_key = os.environ['OPENAI_API_KEY']
client = OpenAI(api_key=api_key)

def generate_example(prompt, prev_examples, temperature=0.5):
    messages = [
        {
            "role": "system",
            "content": f"You are generating data which will be used to train a machine learning model.\n\nYou will be given a high-level description of the model we want to train, and from that, you will generate data samples, each with a prompt/response pair.\n\nYou will do so in this format:\n```\nprompt\n-----------\n$prompt_goes_here\n-----------\n\nresponse\n-----------\n$response_goes_here\n-----------\n```\n\nOnly one prompt/response pair should be generated per turn.\n\nFor each turn, make the example slightly more complex than the last, while ensuring diversity.\n\nMake sure your samples are unique and diverse, yet high-quality and complex enough to train a well-performing model.\n\nHere is the type of model we want to train:\n`{prompt}`"
        }
    ]

    if len(prev_examples) > 0:
        if len(prev_examples) > 8:
            prev_examples = random.sample(prev_examples, 8)
        for example in prev_examples:
            messages.append({
                "role": "assistant",
                "content": example
            })

    response: Completion = client.chat.completions.create(
        model="gpt-4",
        messages=messages,
        temperature=temperature,
        max_tokens=1000,
    )

    return response.choices[0].message.content

# Generate examples
prev_examples = []

for i in range(number_of_examples):
    print(f'Generating example {i}')
    example = generate_example(prompt, prev_examples)
    prev_examples.append(example)

print(prev_examples)


Generating example 0
prompt
-----------
Hey, I'm doing my math homework and I'm stuck on a problem. It says, "A train travels at a speed of 60 miles per hour. How long will it take to travel 240 miles?" I don't understand how to solve it. Can you help me?
-----------

response
-----------
Of course, I'd be happy to help! The problem is asking you to find the time it takes for the train to travel a certain distance. In this case, you can use the formula for time which is distance divided by speed. So, you want to divide the total distance, which is 240 miles, by the speed of the train, which is 60 miles per hour. What do you get when you do that calculation?

Generating example 1
prompt
-----------
I need help with my English homework. I have to find the main idea of a paragraph but I don't know how to do that. Can you explain it to me?
-----------

response
-----------
Absolutely, I'd be glad to help. The main idea of a paragraph is what the paragraph is mostly about. It's the central 

We also need to generate a system message.

In [ ]:
def generate_system_message(prompt):

    response: Completion = client.chat.completions.create(
        model="gpt-4",
        messages=[
          {
            "role": "system",
            "content": "You will be given a high-level description of the model we are training, and from that, you will generate a simple system prompt for that model to use. Remember, you are not generating the system message for data generation -- you are generating the system message to use for inference. A good format to follow is `Given $INPUT_DATA, you will $WHAT_THE_MODEL_SHOULD_DO.`.\n\nMake it as concise as possible. Include nothing but the system prompt in your response.\n\nFor example, never write: `\"$SYSTEM_PROMPT_HERE\"`.\n\nIt should be like: `$SYSTEM_PROMPT_HERE`."
          },
          {
              "role": "user",
              "content": prompt.strip(),
          }
        ],
        temperature=temperature,
        max_tokens=500,
    )

    return response.choices[0].message.content

system_message = generate_system_message(prompt)

print(f'The system message is: `{system_message}`. Feel free to re-run this cell if you want a better result.')

The system message is: `Given a homework question, guide the student through the problem-solving process without directly providing the answer, ensuring to explain any related concepts in a comprehensive and tutor-like manner.`. Feel free to re-run this cell if you want a better result.


Now let's put our examples into a dataframe and turn them into a final pair of datasets.

In [ ]:
import json
import pandas as pd

# Initialize lists to store prompts and responses
prompts = []
responses = []

# Parse out prompts and responses from examples
for example in prev_examples:
  try:
    split_example = example.split('-----------')
    prompts.append(split_example[1].strip())
    responses.append(split_example[3].strip())
  except:
    pass

# Create a DataFrame
df = pd.DataFrame({
    'prompt': prompts,
    'response': responses
})

# Remove duplicates
df = df.drop_duplicates()

print('There are ' + str(len(df)) + ' successfully-generated examples.')

# Initialize list to store training examples
training_examples = []

# Create training examples in the format required for GPT-3.5 fine-tuning
for index, row in df.iterrows():
    training_example = {
        "messages": [
            {"role": "system", "content": system_message.strip()},
            {"role": "user", "content": row['prompt']},
            {"role": "assistant", "content": row['response']}
        ]
    }
    training_examples.append(training_example)

# Save training examples to a .jsonl file
with open('training_examples.jsonl', 'w') as f:
    for example in training_examples:
        f.write(json.dumps(example) + '\n')

There are 25 successfully-generated examples.


# Upload the file to OpenAI

In [ ]:
file_id = client.files.create(
  file=open("/content/training_examples.jsonl", "rb"),
  purpose='fine-tune'
).id

# file_creation_response = File.create(
#     file=open("/content/training_examples.jsonl", "rb"),
#     purpose='fine-tune'
# )
# file_id = file_creation_response.id

# Train the model! You may need to wait a few minutes before running the next cell to allow for the file to process on OpenAI's servers.

In [ ]:
job = client.fine_tuning.jobs.create(training_file=file_id, model="gpt-3.5-turbo")

job_id = job.id

# Now, just wait until the fine-tuning run is done, and you'll have a ready-to-use model!

Run this cell every 20 minutes or so -- eventually, you'll see a message "New fine-tuned model created: ft:gpt-3.5-turbo-0613:xxxxxxxxxxxx"

Once you see that message, you can go to the OpenAI Playground (or keep going to the next cells and use the API) to try the model!

In [ ]:
client.fine_tuning.jobs.list_events(job_id, limit=10)

SyncCursorPage[FineTuningJobEvent](data=[FineTuningJobEvent(id='ftevent-82m2ONoIN5Ub1NjXoYPfHvK4', created_at=1701536859, level='info', message='The job has successfully completed', object='fine_tuning.job.event', data={}, type='message'), FineTuningJobEvent(id='ftevent-Sd4fhX5PFW4etwifeeOZ3LO8', created_at=1701536856, level='info', message='New fine-tuned model created: ft:gpt-3.5-turbo-0613:episcopal-academy::8RNkmDgh', object='fine_tuning.job.event', data={}, type='message'), FineTuningJobEvent(id='ftevent-28hTsArdBZA8Ap0PbASCB9Gd', created_at=1701536837, level='info', message='Step 91/100: training loss=0.22', object='fine_tuning.job.event', data={'step': 91, 'train_loss': 0.21528035402297974, 'train_mean_token_accuracy': 0.9354838728904724}, type='metrics'), FineTuningJobEvent(id='ftevent-mNdMshytqmA5JZteOKs2Uewt', created_at=1701536821, level='info', message='Step 81/100: training loss=0.37', object='fine_tuning.job.event', data={'step': 81, 'train_loss': 0.37145909667015076, 'tr

# Once your model is trained, run the next cell to grab the fine-tuned model name.

In [ ]:
model_name_pre_object = client.fine_tuning.jobs.retrieve(job_id)
model_name = model_name_pre_object.fine_tuned_model
print(model_name)

ft:gpt-3.5-turbo-0613:episcopal-academy::8RNkmDgh


# Let's try it out!

In [ ]:
response = openai.ChatCompletion.create(
    model=model_name,
    messages=[
      {
        "role": "system",
        "content": system_message,
      },
      {
          "role": "user",
          "content": df['prompt'].sample().values[0],
      }
    ],
)

response.choices[0].message['content']